In [34]:
import pandas as pd                                                               # leitura e processamento de datasets
from sklearn.model_selection import train_test_split                              # modelos estatísticos para aplicar aos dados
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             classification_report,
                             mean_squared_error)
import matplotlib.pyplot as plt

import numpy as np                                                                # manipulação de arrays multidimensionais
import matplotlib.pyplot as plt                                                   # visualização de dados
from sklearn.linear_model import LinearRegression

In [2]:
# Dados com 0 e 1
data = {'idade': [25,35,45,20,30,50,40,55,60,35],
        'renda': [3000,4000,5000,2000,3500,6000,5500,7000,8000,9000],
        'compra': [0, 0, 1, 0, 1, 1, 1, 1, 1, 1]
}

data = pd.DataFrame(data)

In [3]:
# Reverter 0/1 para Nao/Sim manualmente
data['compra'] = ['Nao' if x==0 else 'Sim' for x in data['compra']]

In [25]:
# Separar features e target
X = data[['idade', 'renda']]          # FEATURES (variáveis independentes)
y = data['compra']                    # TARGET (variáveis dependente)

# Divisão treino/teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.5,
                                                        random_state=42)

# Instanciar e treinar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_treino, y_treino)

X_treino

,idade,renda
2,45,5000
9,35,9000
4,30,3500
3,20,2000
6,40,5500


In [ ]:
# 1. Coeficientes
'''
Cada coeficiente corresponde a uma variável independente do modelo.
    Quanto >, mais afeta a variável dependente (compra), quanto <, menos afeta.
'''
print("Coeficientes:", modelo.coef_)

'''
É o valor do logit quando todas as variáveis independentes são zero.
    Indica que, sem renda e idade, a chance de "Sim" seria x.
'''
print("Intercepto:", modelo.intercept_)



In [28]:
# 2. Previsões nos dados de teste
'''
  São as previsões do modelo para cada observação no conjunto de teste.
'''
previsoes = modelo.predict(X_teste)
print("Previsões:", previsoes)

# Previsão para um novo cliente
novo_cliente = pd.DataFrame({'idade': [int(input('Digite a idade: '))],
                             'renda': [int(input('Digite a renda: '))]})
previsao_novo = modelo.predict(novo_cliente)
print("\nPrevisão para novo cliente:\n", previsao_novo)

Previsões: ['Sim' 'Sim' 'Sim' 'Sim' 'Sim']
Digite a idade: 4000
Digite a renda: 4000

Previsão para novo cliente:
 ['Sim']


In [29]:
# 3. Acurácia (ou Score)
'''
  Mede a proporção de previsões corretas do modelo em relação
    ao total de previsões feitas. Quanto maior, melhor.
                  acurácia = número de previsões corretas / total de previsões
'''
acuracia = accuracy_score(y_teste, previsoes)
print("Acurácia:", acuracia)

Acurácia: 0.6


In [ ]:
# 4. Matriz de confusão
'''
  TN (True Negative): reais "Nao" que o modelo previu "Nao"
  FP (False Positive): reais "Nao" que o modelo previu "Sim"
  FN (False Negative): reais "Sim" que o modelo previu "Nao"
  TP (True Positive): reais "Sim" que o modelo previu "Sim"

                        [[TN FP]
                        [FN  TP]]

              Acurácia = TP + TN / TP + TN + FP + FN
'''

matrix = confusion_matrix(y_teste, previsoes)
print("Matriz de confusão:\n", matrix)


In [33]:
# 5. Relatório de classificação
'''
  Precisão = TP / (TP + FP)                                                       -> Proporção de previsões corretas de uma classe em relação ao total de previsões para essa classe;
  Recall = TP / (TP + FN)                                                         -> Proporção de casos reais de uma classe que foram corretamente previstos pelo modelo;
  F1-score = 2 * (precisão * recall) / (precisão + recall)                        -> Média harmônica entre precision e recall. Serve para equilibrar precisão e recall
                                                                                        pois nem só a acurácia nem só a precisão contam a história toda;


'''


print("Relatório de classificação:\n", classification_report(y_teste, previsoes,
                                                             zero_division=0))


Relatório de classificação:
               precision    recall  f1-score   support

         Nao       0.00      0.00      0.00         2
         Sim       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.30      0.50      0.38         5
weighted avg       0.36      0.60      0.45         5



In [ ]:
# Criar meshgrid
x_min, x_max = X['idade'].min() - 5, X['idade'].max() + 5       # espaços maiores q o máximo
y_min, y_max = X['renda'].min() - 500, X['renda'].max() + 500   # & menores q o mínimo

xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))


# Prever probabilidades de "Sim"
Z_prob = modelo.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:,1]
Z_prob = Z_prob.reshape(xx.shape)

# Plotar o gradiente de probabilidades
plt.figure(figsize=(10,6))
contour = plt.contourf(xx, yy, Z_prob, 20, cmap='RdYlBu', alpha=0.5)
plt.colorbar(contour, label='Probabilidade de compra (Sim)')

# Plotar linha de decisão (probabilidade = 0.5)
plt.contour(xx, yy, Z_prob, levels=[0.5], colors='green', linewidths=2,
            linestyles='--')

# Plotar pontos reais
cores = {'Nao':'red', 'Sim':'blue'}
for classe in data['compra'].unique():
    subset = data[data['compra']==classe]
    plt.scatter(subset['idade'], subset['renda'],
                color=cores[classe], label=classe,
                s=100, edgecolor='k')

plt.xlabel('Idade')
plt.ylabel('Renda')
plt.title('Regressão Logística: Probabilidade de Compra')
plt.legend()
plt.show()


In [ ]:
# Dados históricos
anos = np.array([2019, 2020, 2021, 2022, 2023, 2024, 2025]).reshape(-1, 1)
ppg = np.array([10.8, 19.0, 23.7, 24.5, 31.4, 30.1, 32.7])

# Modelo de regressão linear
modelo = LinearRegression()
modelo.fit(anos, ppg)

# Previsão para 2026
ano_futuro = np.array([[2026]])
ppg_previsto = modelo.predict(ano_futuro)

# Exibindo o resultado
print(f"Projeção para 2025–26: {ppg_previsto[0]:.2f} pontos por jogo")

# Visualizando
plt.scatter(anos, ppg, color='blue', label='Dados Reais')
plt.plot(anos, modelo.predict(anos), color='red', label='Tendência Linear')
plt.scatter(2026, ppg_previsto, color='green', label='Projeção 2026', zorder=5)
plt.xlabel('Ano')
plt.ylabel('Pontos por Jogo')
plt.title('Projeção de Pontos por Jogo de Shai Gilgeous-Alexander')
plt.legend()
plt.grid(True)
plt.show()
